# State Preparation - Complete Guide for Qiskit Certification

**Section 1: Quantum Operations - Part 3**

This notebook covers quantum state preparation and circuit control operations.

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Use `initialize()` for arbitrary state preparation
- Understand `barrier()` for circuit organization
- Apply `reset()` for qubit recycling
- Use `delay()` for timing control
- Master common state preparation patterns

### 🧠 Key Exam Mnemonics

**"Barriers Block, Reset Returns"**
- `barrier()` = Visual only, blocks optimization
- `reset()` = Active operation, returns to |0⟩

**Initialize vs Reset**:
- `initialize()`: Synthesis (adds many gates)
- `reset()`: Measurement-based (active reset)

---

## Setup

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_state_city
import matplotlib.pyplot as plt

%matplotlib inline

print("✅ All imports successful!")

---

## 1. Initialize() - Arbitrary State Preparation

Create quantum states directly from state vectors

### PURPOSE of initialize()

**What it does**: Sets qubit to specific quantum state (arbitrary superposition)

**When to use**:
- Algorithm initialization (Grover, VQE, etc.)
- Preparing training data for quantum machine learning
- Setting up specific test states

**How it works**:
- Decomposes target state into sequence of gates
- Automatically normalizes if amplitudes don't sum to 1
- Can initialize single or multiple qubits

**Exam Fact**: `initialize()` synthesizes a circuit - not a single gate!

---

In [ ]:
# Initialize to |1⟩
qc = QuantumCircuit(1)
qc.initialize([0, 1], 0)

print("Initialize to |1⟩")
print("=" * 50)
display(qc.draw(output='mpl'))

state = Statevector(qc)
print(f"\nState: {state.data}")
plot_bloch_multivector(state)

In [ ]:
# Initialize to |+⟩
qc = QuantumCircuit(1)
qc.initialize([1/np.sqrt(2), 1/np.sqrt(2)], 0)

print("Initialize to |+⟩")
print("=" * 50)
display(qc.draw(output='mpl'))

state = Statevector(qc)
print(f"\nState: {state.data}")
plot_bloch_multivector(state)

In [ ]:
# Initialize 2-qubit Bell state
qc = QuantumCircuit(2)
qc.initialize([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)], [0, 1])

print("Initialize 2-qubit Bell state")
print("=" * 50)
display(qc.draw(output='mpl'))

state = Statevector(qc)
print(f"\nState: {state.data}")
print("Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2")

plot_state_city(state)

**🎯 EXAM TIP**: 
- initialize() creates preparation circuit
- Useful for algorithm initialization
- Automatically normalizes if needed

---

## 2. Barrier() - Circuit Alignment

Visual separator and optimization control

### PURPOSE of barrier()

**What it does**: Visual separator that prevents optimization across boundary

**When to use**:
- Organize circuits into logical sections
- Prevent transpiler from optimizing away gates
- Align operations across multiple qubits
- Debug: see clear circuit structure

**How it works**:
- Does NOT change quantum state
- Only affects transpilation/optimization
- Acts as "fence" for gate reordering

**Exam Fact**: `barrier()` is visual only - doesn't appear in final operations!

---

In [ ]:
# Circuit with barriers
qc = QuantumCircuit(3)
qc.h(range(3))
qc.barrier()  # Visual separator
qc.cx(0, 1)
qc.cx(1, 2)
qc.barrier()
qc.h(range(3))

print("Circuit with barriers:")
display(qc.draw(output='mpl'))

**🎯 EXAM TIP**: 
- barrier() doesn't affect state, only visualization
- Prevents transpiler optimization across barrier
- Good for organizing circuit sections

---

## 3. Reset() - Reset to |0⟩

Forces qubit to ground state

### PURPOSE of reset()

**What it does**: Returns qubit to |0⟩ state (measurement + conditional flip)

**When to use**:
- Qubit recycling in limited hardware
- Mid-circuit resets in error correction
- Clearing ancilla qubits for reuse
- Dynamic circuits

**How it works**:
1. Measures qubit
2. If result is |1⟩, applies X gate
3. Guarantees qubit is in |0⟩

**Exam Fact**: `reset()` uses measurement - different from `initialize([1,0], q)`!

### ⚠️ EXAM TRAP: Initialize vs Reset

| Feature | initialize() | reset() |
|---------|-------------|---------|
| Method | Gate synthesis | Measurement-based |
| Speed | Slower (many gates) | Faster (measure + flip) |
| Use case | Arbitrary states | Return to |0⟩ only |
| Circuit depth | Deep | Shallow |

---

In [ ]:
# Reset qubit
qc = QuantumCircuit(2)
qc.x([0, 1])  # Both qubits to |1⟩
qc.reset(0)   # Reset first qubit

print("Reset demonstration:")
display(qc.draw(output='mpl'))

state = Statevector(qc)
print(f"\nFinal state: {state.data}")
print("Qubit 0 reset to |0⟩, qubit 1 remains |1⟩")
print("Result: |01⟩")

**🎯 EXAM TIP**: 
- reset() forces qubit to |0⟩
- Useful for qubit recycling
- Important for limited hardware

---

## 4. Delay() - Timing Control

For pulse-level timing control

In [ ]:
# Circuit with delay
qc = QuantumCircuit(2)
qc.h(0)
qc.delay(100, 0, unit='ns')  # 100 nanoseconds
qc.cx(0, 1)

print("Circuit with delay:")
display(qc.draw(output='mpl'))

print("\n100 ns delay on qubit 0 between H and CNOT")

In [ ]:
# Different time units
qc = QuantumCircuit(1)
qc.h(0)
qc.delay(100, 0, unit='ns')   # nanoseconds
qc.delay(1, 0, unit='us')     # microseconds
qc.delay(1, 0, unit='ms')     # milliseconds

print("Different delay units:")
display(qc.draw(output='mpl'))

In [ ]:
# Verify gate equivalences
print("GATE EQUIVALENCES - State Preparation")
print("=" * 60)

# H|0⟩ ≈ RY(π/2)|0⟩
qc_h = QuantumCircuit(1)
qc_h.h(0)
state_h = Statevector(qc_h)

qc_ry = QuantumCircuit(1)
qc_ry.ry(np.pi/2, 0)
state_ry = Statevector(qc_ry)

print("1. H|0⟩ vs RY(π/2)|0⟩")
print(f"   H|0⟩     = {state_h.data}")
print(f"   RY(π/2)|0⟩ = {state_ry.data}")
print(f"   Close enough: {np.allclose(np.abs(state_h.data), np.abs(state_ry.data))} ✓")

# RX(π) = X
qc_rx_pi = QuantumCircuit(1)
qc_rx_pi.rx(np.pi, 0)
state_rx_pi = Statevector(qc_rx_pi)

qc_x = QuantumCircuit(1)
qc_x.x(0)
state_x = Statevector(qc_x)

print("\n2. RX(π)|0⟩ = X|0⟩")
print(f"   RX(π)|0⟩ = {state_rx_pi.data}")
print(f"   X|0⟩     = {state_x.data}")
print(f"   Equal: {np.allclose(state_rx_pi.data, state_x.data, atol=1e-10)} ✓")


---

## 6. Gate Equivalences for State Preparation

**These relationships appear in exam questions!**

```
X = RX(π)              (180° rotation = bit flip)
H|0⟩ = RY(π/2)|0⟩     (Hadamard ≈ Y-rotation by 90°)
initialize([1,0]) ≈ reset()  (Both give |0⟩, different methods)
```

**Exam Pattern**: "Which operation prepares state |+⟩?"
- Answer: H|0⟩ (most common)
- Alternative: RY(π/2)|0⟩
- Alternative: initialize([1/√2, 1/√2], 0)

---

In [ ]:
# Pattern 1: Single-qubit superposition
print("Pattern 1: Single-Qubit Superposition")
print("=" * 60)

qc_p1 = QuantumCircuit(1)
qc_p1.h(0)
state_p1 = Statevector(qc_p1)
print(f"H|0⟩ = {state_p1.data} = |+⟩")

# Pattern 2: Multi-qubit superposition
print("\nPattern 2: Multi-Qubit Superposition (n=3)")
print("=" * 60)

qc_p2 = QuantumCircuit(3)
for i in range(3):
    qc_p2.h(i)

state_p2 = Statevector(qc_p2)
print(f"H⊗H⊗H|000⟩ creates superposition of all 2^3 = 8 states")
print(f"State vector (first 8 components):")
for i in range(8):
    print(f"|{i:03b}⟩: {state_p2.data[i]:.4f}")

display(qc_p2.draw(output='mpl'))


---

## 5. Common State Preparation Patterns - EXAM ESSENTIAL

### Pattern 1: Single-Qubit Superposition
```python
# Create |+⟩ = (|0⟩+|1⟩)/√2
qc.h(0)

# Alternative with initialize
qc.initialize([1/√2, 1/√2], 0)
```

### Pattern 2: Multi-Qubit Superposition
```python
# Create all 2^n states equally
for i in range(n):
    qc.h(i)
# Result: (|00...0⟩ + |00...1⟩ + ... + |11...1⟩) / √(2^n)
```

**Exam Fact**: Applying H to n qubits creates superposition of ALL 2^n states!

### Pattern 3: Specific Multi-Qubit State
```python
# Create |Ψ⟩ = α|00⟩ + β|01⟩ + γ|10⟩ + δ|11⟩
amplitudes = [α, β, γ, δ]
qc.initialize(amplitudes, [0, 1])
```

---

**🎯 EXAM TIP**: 
- delay() for pulse-level control
- Units: ns (nanoseconds), us (microseconds), ms (milliseconds)
- Used in timing-sensitive experiments

---

## 5. Practical Examples

Combining state preparation and control operations

In [ ]:
# Example: Initialize, manipulate, reset
qc = QuantumCircuit(3)

# Initialize first qubit to |+⟩
qc.initialize([1/np.sqrt(2), 1/np.sqrt(2)], 0)

qc.barrier()

# Create entanglement
qc.cx(0, 1)
qc.cx(0, 2)

qc.barrier()

# Reset qubit 1
qc.reset(1)

print("Complex state preparation example:")
display(qc.draw(output='mpl'))

In [ ]:
# Example: Custom initialization for VQE
theta = np.pi / 4

qc = QuantumCircuit(2)

# Initialize to custom angles
qc.ry(theta, 0)
qc.ry(theta, 1)

qc.barrier()

# Entangling layer
qc.cx(0, 1)

qc.barrier()

# Another rotation layer
qc.ry(theta, 0)
qc.ry(theta, 1)

print("VQE-style ansatz with barriers:")
display(qc.draw(output='mpl'))

state = Statevector(qc)
plot_state_city(state)

---

## ✅ Key Takeaways - Master These for the Exam!

### Core Concepts
1. **initialize()** - Arbitrary state preparation (gate synthesis)
2. **barrier()** - Visual separator, prevents optimization
3. **reset()** - Returns qubit to |0⟩ (measurement-based)
4. **delay()** - Timing control for pulse-level programming

### Critical Exam Facts
- ✅ `initialize()` synthesizes circuit, `reset()` uses measurement
- ✅ `barrier()` doesn't change state, only affects transpilation
- ✅ H^⊗n creates superposition of all 2^n basis states
- ✅ `reset()` is faster than `initialize([1,0])` for returning to |0⟩
- ✅ State preparation is FIRST step in most quantum algorithms

### Mnemonics to Remember
- 🧠 "Barriers Block, Reset Returns"
- 🧠 Initialize = Synthesis, Reset = Measurement

### Common Patterns
- Create |+⟩: `qc.h(0)`
- Create all states: `for i in range(n): qc.h(i)`
- Clear ancilla: `qc.reset(ancilla)`
- Organize circuit: `qc.barrier()`

### What's Next?
**Section 2: Visualization** - Learn to visualize quantum states and circuits

---

## 🎓 Exam Preparation Checklist

Before moving on, make sure you can:
- [ ] Explain difference between `initialize()` and `reset()`
- [ ] Know when to use `barrier()`
- [ ] Create multi-qubit superposition with H gates
- [ ] Understand PURPOSE of each operation
- [ ] Recognize state preparation patterns in algorithms

**If you checked all boxes, you're ready for visualization! 🚀**

---

## 📚 Section 1 Complete!

You've now mastered:
1. ✅ Single-qubit gates (X, Y, Z, H, S, T, rotations)
2. ✅ Multi-qubit gates (CNOT, CZ, SWAP, Toffoli)
3. ✅ Bell states and entanglement
4. ✅ State preparation operations

**Congratulations! These are the foundational building blocks for ALL quantum algorithms! 🎉**


---

## 6. State Preparation Comparison

In [ ]:
# Compare different ways to create |+⟩ state

# Method 1: Hadamard
qc1 = QuantumCircuit(1)
qc1.h(0)

# Method 2: Initialize
qc2 = QuantumCircuit(1)
qc2.initialize([1/np.sqrt(2), 1/np.sqrt(2)], 0)

# Method 3: RY rotation
qc3 = QuantumCircuit(1)
qc3.ry(np.pi/2, 0)

print("Three ways to create |+⟩ state:")
print("\n1. Hadamard:")
display(qc1.draw(output='mpl'))

print("\n2. Initialize:")
display(qc2.draw(output='mpl'))

print("\n3. RY(π/2):")
display(qc3.draw(output='mpl'))

# Verify all give same result
state1 = Statevector(qc1)
state2 = Statevector(qc2)
state3 = Statevector(qc3)

print("\nAll methods produce |+⟩ state:")
print(f"H gate:      {state1.data}")
print(f"Initialize:  {state2.data}")
print(f"RY(π/2):     {state3.data}")

---

## ✅ Key Takeaways

1. **initialize()** creates arbitrary quantum states
2. **barrier()** organizes circuits (no state change)
3. **reset()** forces qubit to |0⟩
4. **delay()** controls timing (pulse-level)
5. Multiple ways to prepare common states

**🎓 Practice state preparation for variational algorithms!**